
# --- Boltz2: Protein Structure Prediction Pipeline ---

![Python](https://img.shields.io/badge/Python-3.10-blue?logo=python)
![CUDA](https://img.shields.io/badge/CUDA-Enabled-green?logo=nvidia)
![Boltz2](https://img.shields.io/badge/Model-Boltz2-purple)
![Platform](https://img.shields.io/badge/Platform-Colab%20|%20Linux-lightgrey?logo=googlecolab)
![License](https://img.shields.io/badge/License-MIT-orange)
![Status](https://img.shields.io/badge/Status-Active-success)
![Build](https://img.shields.io/badge/Build-Stable-brightgreen)
![Contributions](https://img.shields.io/badge/Contributions-Welcome-blue)
<br>

---

## Boltz2: Deep Learning Pipeline for Protein Structure Prediction

Boltz2 is an **open-source, deep learning-based software** for predicting **3D protein structures** from amino acid sequences.  
It leverages **advanced neural networks** and **diffusion models** to generate accurate protein models, supporting both **monomeric** and **complex assemblies**.

---

###  Pipeline Overview
1. **Input**: Provide a protein sequence (and optional ligands).  
2. **YAML Generation**: The sequence is formatted into a YAML config.  
3. **MSA Search**: Boltz2 fetches multiple sequence alignments (MSA) using online servers.  
4. **Structure Prediction**: The neural network predicts 3D coordinates using diffusion and recycling steps.  
5. **Output**: Results include 3D models (CIF/PDB), confidence scores (**pLDDT**), and error heatmaps (**PAE**).  
6. **Visualization**: The notebook displays the predicted structure and confidence plots.  

---

 **Note:** This notebook automates the full Boltz2 workflow, from setup to visualization, with **color-coded status** and **interactive outputs**.  

---

##  Credits & Authorship

- **Notebook Developer:** Atharva Tilewale  
- **Affiliation:** Gujarat Biotechnology University | Bioinformatics & Computational Biology  
- **GitHub Repository:** [Boltz-Notebook](https://github.com/AtharvaTilewale/Boltz-Notebook)  
- **Contact:** [LinkedIn](https://www.linkedin.com/in/atharvatilewale) | [GitHub](https://github.com/AtharvaTilewale)  

**Acknowledgements:**  
- **Boltz2 framework**: [Original Boltz repository](https://github.com/jwohlwend/boltz) by J. Wohlwend and collaborators.  
- **Dependencies:** PyTorch, Biopython, NumPy, Matplotlib, Py3Dmol, PyYAML.  
- Special thanks to the **open-source community** for providing tools that make structural bioinformatics more accessible.  

---

## References

- Passaro, S., Corso, G., Wohlwend, J., Reveiz, M., Thaler, S., Somnath, V. R., Getz, N., Portnoi, T., Roy, J., Stark, H., Kwabi-Addo, D., Beaini, D., Jaakkola, T., & Barzilay, R. (2025).  
  **Boltz-2: Towards Accurate and Efficient Binding Affinity Prediction.** *bioRxiv.*  
    [![bioRxiv Boltz2](https://img.shields.io/badge/bioRxiv-Boltz2-red)](https://doi.org/10.1101/2025.06.14.659707)

- Wohlwend, J., Corso, G., Passaro, S., Getz, N., Reveiz, M., Leidal, K., Swiderski, W., Atkinson, L., Portnoi, T., Chinn, I., Silterra, J., Jaakkola, T., & Barzilay, R. (2024).  
  **Boltz-1: Democratizing Biomolecular Interaction Modeling.** *bioRxiv.*  
    [![bioRxiv Boltz1](https://img.shields.io/badge/bioRxiv-Boltz1-orange)](https://doi.org/10.1101/2024.11.19.624167)

- Mirdita, M., Schütze, K., Moriwaki, Y., Heo, L., Ovchinnikov, S., & Steinegger, M. (2022).  
  **ColabFold: Making protein folding accessible to all.** *Nature Methods.*  
    [![ColabFold](https://img.shields.io/badge/ColabFold-Reference-yellow)](https://doi.org/10.1038/s41592-022-01488-1)

---

## Cite
If you use this notebook, please **cite the following repository**:

[![GitHub Repo](https://img.shields.io/badge/GitHub-Boltz--Notebook-181717?logo=github)](https://github.com/AtharvaTilewale/Boltz-Notebook)

In [ ]:
# @title Install Dependencies and Boltz2 with CUDA support
import sys
import subprocess
import threading
import time
import os
import shutil

# ANSI color codes for colored output
class Color:
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    RESET = "\033[0m"

repo_dir = "boltz"

steps = [
    {
        "loader": f"{Color.CYAN}Cloning repository...{Color.RESET}",
        "done": f"[{Color.GREEN}✔{Color.RESET}] Repository cloned successfully.",
        "fail": f"[{Color.RED}✘{Color.RESET}] Repository clone failed.",
        "cmd": ["git", "clone", "https://github.com/jwohlwend/boltz.git"]
    },
    {
        "loader": f"{Color.RESET}Installing dependencies...{Color.RESET}",
        "done": f"[{Color.GREEN}✔{Color.RESET}] Dependencies installed successfully.",
        "fail": f"[{Color.RED}✘{Color.RESET}] Dependency installation failed.",
        "cmd": [sys.executable, "-m", "pip", "install", "-e", "boltz[cuda]", "biopython", "numpy", "matplotlib", "pyyaml", "py3Dmol", "git+https://github.com/AtharvaTilewale/Boltz-Notebook.git", "--quiet"]
    },
    {
        "loader": f"{Color.CYAN}Validating installation...{Color.RESET}",
        "done": f"[{Color.GREEN}✔{Color.RESET}] Validation complete.",
        "fail": f"[{Color.RED}✘{Color.RESET}] Validation failed.",
        "cmd": [sys.executable, "-c", "import torch; print('Torch CUDA available:', torch.cuda.is_available()); print('CUDA device count:', torch.cuda.device_count())"]
    }
]

def loader(msg, stop_event):
    symbols = ["-", "\\", "|", "/"]
    i = 0
    while not stop_event.is_set():
        sys.stdout.write(f"\r[{symbols[i % len(symbols)]}] {msg}   ")
        sys.stdout.flush()
        time.sleep(0.1)
        i += 1
    sys.stdout.write("\r" + " " * (len(msg) + 10) + "\r")

# Step 1: Remove repo if it exists
if os.path.isdir(repo_dir):
    print(f"{Color.YELLOW}[i] Repository already exists. Removing '{repo_dir}'...{Color.RESET}")
    try:
        shutil.rmtree(repo_dir)
        print(f"[{Color.GREEN}✔{Color.RESET}] Existing repository removed.")
    except Exception as e:
        print(f"[{Color.RED}✘{Color.RESET}] Failed to remove existing repository: {e}")
        raise

all_success = True

# Main steps
for step in steps:
    stop_event = threading.Event()
    t = threading.Thread(target=loader, args=(step["loader"], stop_event))
    t.start()
    try:
        subprocess.run(step["cmd"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        stop_event.set()
        t.join()
        print(step["done"])
    except Exception as e:
        stop_event.set()
        t.join()
        print(f"{step['fail']} {e}")
        all_success = False
        break

if all_success:
    print(f"{Color.GREEN}All steps completed successfully.{Color.RESET}")
    from logger import log_event
    log_event("Done")


In [ ]:

# @title Download CCD Dataset and Test Boltz2
import sys
import threading
import time
import os

# ANSI color codes for colored output
class Color:
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    RESET = "\033[0m"

def loader(msg, stop_event):
    symbols = ["-", "\\", "|", "/"]
    i = 0
    while not stop_event.is_set():
        sys.stdout.write(f"\r[{symbols[i % len(symbols)]}] {msg}   ")
        sys.stdout.flush()
        time.sleep(0.1)
        i += 1
    sys.stdout.write("\r" + " " * (len(msg) + 10) + "\r")
    sys.stdout.flush()

# Step 1: Create data directory
os.makedirs("/content/boltz_data", exist_ok=True)
os.chdir("/content/boltz_data/")

# Step 2: Write YAML file
yaml_content = f"""\
version: 1
sequences:
    - protein:
        id: [A]
        sequence: MVTPEGNVSLVDESLLVGVTDEDRAVRSAHQF
    - ligand:
        id: [B]
        ccd: SAH   # fetch ligand from CCD
    - ligand:
        id: [C]
        smiles: 'N[C@@H](Cc1ccc(O)cc1)C(=O)O'
"""
with open("/content/boltz_data/test.yaml", "w") as f:
    f.write(yaml_content)

# Step 3: Run boltz predict (silent)
step_msg = f"{Color.YELLOW}Downloading CCD Dataset...{Color.RESET}"
stop_event = threading.Event()
t = threading.Thread(target=loader, args=(step_msg, stop_event))
t.start()
try:
    import subprocess
    subprocess.run(
        ["boltz", "predict", "test.yaml", "--use_msa_server"],
        cwd="/content/boltz_data",
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        check=True
    )
    stop_event.set()
    t.join()
    print(f"[{Color.GREEN}✔{Color.RESET}] CCD Dataset Downloaded and validated.")
except Exception as e:
    stop_event.set()
    t.join()
    print(f"[{Color.RED}✘{Color.RESET}] CCD Dataset Download or validation failed: {e}")


In [ ]:
# @title Generate Parameters (YAML file & Run Config)
# Colab HTML UI -> Python File Savers
from IPython.display import HTML, display
import yaml
from google.colab import output
import os
import re

# Ensure the directory exists before changing into it
if not os.path.exists("/content/boltz_data/"):
    os.makedirs("/content/boltz_data/")
os.chdir("/content/boltz_data/")

# --- START: Custom YAML Formatting (No changes needed here) ---
class IdList(list): pass

def represent_id_list(dumper, data):
    return dumper.represent_sequence('tag:yaml.org,2002:seq', data, flow_style=True)

def str_presenter(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

class MyDumper(yaml.SafeDumper):
    pass

class QuotedString(str): pass

def quoted_str_presenter(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style="'")

MyDumper.add_representer(QuotedString, quoted_str_presenter)
MyDumper.add_representer(IdList, represent_id_list)
MyDumper.add_representer(str, str_presenter)
# --- END: Custom YAML Formatting ---

def _save_params(data):
    if not isinstance(data, dict) or 'sequences' not in data:
        return {'status': 'error', 'message': 'Invalid data structure: "sequences" key missing.'}

    sequences_fixed = []
    for entry in data['sequences']:
        if 'protein' in entry:
            ids = IdList([i.upper().replace(' ', '') for i in entry['protein'].get('id', [])])
            seq = re.sub(r'\s+', '', entry['protein'].get('sequence', '').upper())
            protein_dict = {'id': ids, 'sequence': seq}
            sequences_fixed.append({'protein': protein_dict})
        elif 'ligand' in entry:
            ids = IdList([i.upper().replace(' ', '') for i in entry['ligand'].get('id', [])])
            ligand_dict = {'id': ids}
            if 'ccd' in entry['ligand']:
                ligand_dict['ccd'] = entry['ligand']['ccd'].upper().replace(' ', '')
            if 'smiles' in entry['ligand']:
                smiles_val = entry['ligand']['smiles'].replace(' ', '')
                ligand_dict['smiles'] = QuotedString(smiles_val)
            sequences_fixed.append({'ligand': ligand_dict})

    # Reconstruct the final dictionary to be dumped in the desired order
    final_yaml_data = {'version': 1}
    final_yaml_data['sequences'] = sequences_fixed # Add sequences first
    if 'properties' in data:
        final_yaml_data['properties'] = data['properties'] # Add properties last

    filename = "params.yaml"
    try:
        with open(filename, 'w') as f:
            yaml.dump(
                final_yaml_data,
                f, Dumper=MyDumper, sort_keys=False, default_flow_style=False, indent=2
            )
        return {'status': 'ok', 'filename': filename}
    except Exception as e:
        return {'status': 'error', 'message': str(e)}

def _save_run_params(data):
    try:
        filename = data.get('filename', 'run_params.txt')
        content = data.get('content', '')
        with open(filename, 'w') as f:
            f.write(content)
        return {'status': 'ok'}
    except Exception as e:
        return {'status': 'error', 'message': str(e)}

output.register_callback('save_params', _save_params)
output.register_callback('save_run_params', _save_run_params)

# HTML + JS with a Revamped UI and a second page
html = r"""
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.2/css/all.min.css">
<style>
    /* --- 1. THEME & GLOBAL STYLES --- */
    :root {
        --font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif;
        --primary-color: #3b82f6; --primary-hover: #2563eb;
        --danger-color: #ef4444; --danger-hover: #dc2626;
        --secondary-color: #6b7280; --secondary-hover: #4b5563;
        --success-color: #22c55e; --success-hover: #16a34a;
        --bg-light: #f9fafb; --border-color: #d1d5db;
        --text-dark: #1f2937; --text-light: #4b5563;
        --radius: 8px; --shadow: 0 4px 6px -1px rgba(0,0,0,0.1), 0 2px 4px -2px rgba(0,0,0,0.1);
    }
    @keyframes fadeIn { from { opacity: 0; transform: translateY(-10px); } to { opacity: 1; transform: translateY(0); } }

    /* --- 2. LAYOUT & TYPOGRAPHY --- */
    .container { font-family: var(--font-family); color: var(--text-dark); background: #fff; padding: 24px; }
    .block {
        border: 1px solid var(--border-color); padding: 20px; margin: 16px 0; border-radius: var(--radius);
        background: #fff; box-shadow: var(--shadow); animation: fadeIn 0.4s ease-out; border-top: 4px solid var(--primary-color);
    }
    .block[data-type="ligand"] { border-top-color: #a855f7; }
    .block-header { display:flex; justify-content:space-between; align-items:center; margin-bottom:16px; }
    .title { font-weight: 600; font-size: 1.1em; color: var(--text-dark); display:flex; align-items:center; gap: 8px; }
    .row { display:flex; gap:25px; align-items:center; margin-bottom:20px; }
    .row label { width: 150px; color: var(--text-light); font-size: 0.9em; flex-shrink: 0; display: flex; align-items: center; justify-content: space-between; }
    input[type="checkbox"] { width: auto; flex: 0; height: 16px; width: 16px; cursor: pointer; }
    details { border: 1px solid var(--border-color); border-radius: var(--radius); padding: 12px; margin-top: 20px; }
    summary { font-weight: 500; cursor: pointer; }

    /* --- 3. FORMS & BUTTONS --- */
    input[type="text"], input[type="number"], textarea, select {
        flex: 1; padding: 10px; border: 1px solid var(--border-color); border-radius: 6px;
        font-size: 14px; color: var(--text-dark); background: var(--bg-light); transition: border-color 0.2s, box-shadow 0.2s;
    }
    input[type="text"]:focus, input[type="number"]:focus, textarea:focus, select:focus {
        outline: none; border-color: var(--primary-color); box-shadow: 0 0 0 2px rgba(59, 130, 246, 0.4);
    }
    .btn {
        display: inline-flex; align-items: center; gap: 6px; border: none; padding: 8px 16px;
        border-radius: 6px; cursor: pointer; font-size: 14px; font-weight: 500;
        transition: background-color 0.2s, transform 0.1s;
    }
    .btn:active { transform: scale(0.98); }
    .btn.primary { background: var(--primary-color); color: #fff; }
    .btn.primary:hover { background: var(--primary-hover); }
    .btn.secondary { background: var(--secondary-color); color: #fff; }
    .btn.secondary:hover { background: var(--secondary-hover); }
    .btn.success { background: var(--success-color); color: #fff; }
    .btn.success:hover { background: var(--success-hover); }
    .remove-btn {background: transparent; color: var(--secondary-color); border: none; width: 32px; height: 32px; border-radius: 50%; cursor: pointer; display: inline-flex; align-items: center; justify-content: center; font-size: 1em; transition: background-color 0.2s, color 0.2s;}
    .remove-btn:hover {background-color: #fee2e2; color: var(--danger-color);}

    /* --- 4. TOOLTIPS --- */
    .tooltip-icon {
        position: relative;
        display: inline-block;
        cursor: help;
        color: #ccc;
        border: 1px solid #ccc;
        border-radius: 50%;
        width: 16px;
        height: 16px;
        font-size: 12px;
        line-height: 14px;
        text-align: center;
        font-style: normal;
    }
    .tooltip-icon .tooltip-text {
        visibility: hidden;
        width: 220px;
        background-color: #333;
        color: #fff;
        text-align: left;
        font-size: 0.8em;
        font-weight: 400;
        border-radius: 6px;
        padding: 8px;
        position: absolute;
        z-index: 10;
        bottom: 50%;
        left: 120%;
        transform: translateY(50%);
        opacity: 0;
        transition: opacity 0.3s;
        box-shadow: var(--shadow);
    }
    .tooltip-icon:hover .tooltip-text {
        visibility: visible;
        opacity: 1;
    }


    /* --- 5. CONTROLS & STATUS --- */
    .controls { margin-top: 24px; display:flex; gap:10px; flex-wrap:wrap; align-items:center; }
    .status-message {
        display: flex; align-items: center; gap: 8px; padding: 8px 12px;
        border-radius: 6px; font-size: 0.9em; animation: fadeIn 0.3s;
    }
    .status-message.success { background-color: #dcfce7; color: #166534; }
    .status-message.error { background-color: #fee2e2; color: #991b1b; }
    .status-message.warning { background-color: #fef3c7; color: #92400e; }
</style>

<div class="container">
    <div id="main_params_container">
        <div id="sequences_container"></div>
        <div id="affinity_prediction_container"></div>
        <div class="controls">
            <button class="btn primary" onclick="addProtein()"><i class="fa-solid fa-dna"></i> Add Protein</button>
            <button class="btn primary" style="background-color:#a855f7;" onclick="addLigand()"><i class="fa-solid fa-puzzle-piece"></i> Add Ligand</button>
            <button class="btn secondary" onclick="clearAll()"><i class="fa-solid fa-broom"></i> Clear Added</button>
            <button class="btn secondary" id="saveBtn" onclick="saveYaml()"><i id="saveIcon" class="fa-solid fa-save"></i> <span id="saveBtnText">Save YAML</span></button>
            <button class="btn success" id="nextBtn" onclick="showRunParams()" style="display:none;"><span id="nextBtnText">Next</span> <i class="fa-solid fa-arrow-right"></i></button>
            <div id="status"></div>
        </div>
    </div>

    <div id="run_params_container" style="display:none;">
      <div class="block">
          <div class="title" style="margin-bottom: 20px;"><i class="fa-solid fa-gears"></i> Run Parameters</div>
          <div class="row">
            <label for="rp_job_name">Job Name</label>
            <input type="text" id="rp_job_name" value="Insulin" placeholder="Insulin_Peptide">
          </div>
          <div class="row">
            <label for="rp_use_potentials">Use Potentials <i class="tooltip-icon">?<span class="tooltip-text">Enable the use of pre-computed potentials to guide the generation process.</span></i></label>
            <input type="checkbox" id="rp_use_potentials" checked>
          </div>
          <div class="row">
            <label for="rp_override">Override <i class="tooltip-icon">?<span class="tooltip-text">If a file with the same job name already exists, this option will overwrite it.</span></i></label>
            <input type="checkbox" id="rp_override" checked>
          </div>

          <details>
              <summary>Advanced Options</summary>
              <div style="padding-top: 20px;">
                  <div class="row">
                    <label for="rp_recycling_steps">Recycling Steps <i class="tooltip-icon">?<span class="tooltip-text">Number of times to recycle the output structure back into the model for refinement.</span></i></label>
                    <input type="number" id="rp_recycling_steps" value="3" step="1">
                  </div>
                  <div class="row">
                      <label for="rp_sampling_steps">Sampling Steps <i class="tooltip-icon">?<span class="tooltip-text">Number of steps in the diffusion process. More steps can lead to higher quality but take longer.</span></i></label>
                      <input type="range" id="rp_sampling_steps" min="50" max="400" step="50" value="200" oninput="this.nextElementSibling.value = this.value">
                      <output>200</output>
                  </div>
                  <div class="row">
                    <label for="rp_diffusion_samples">Diffusion Samples <i class="tooltip-icon">?<span class="tooltip-text">Number of independent structures to generate.</span></i></label>
                    <input type="number" id="rp_diffusion_samples" value="1" step="1">
                  </div>
                  <div class="row">
                    <label for="rp_step_scale">Step Scale <i class="tooltip-icon">?<span class="tooltip-text">Controls the noise schedule during diffusion. Higher values can sometimes improve structure quality.</span></i></label>
                    <input type="number" id="rp_step_scale" value="1.638" step="0.1">
                  </div>
                  <div class="row">
                      <label for="rp_max_msa_seqs">Max MSA Sequences<i class="tooltip-icon">?<span class="tooltip-text">The maximum number of sequences to use from the Multiple Sequence Alignment (MSA).</span></i></label>
                      <select id="rp_max_msa_seqs">
                          <option>32</option><option>64</option><option>128</option><option>256</option><option>512</option>
                          <option>1024</option><option>2048</option><option>4096</option><option selected>8192</option>
                      </select>
                  </div>
                  <div class="row">
                    <label for="rp_subsample_msa">Subsample MSA <i class="tooltip-icon">?<span class="tooltip-text">If enabled, a smaller, random subset of the MSA will be used.</span></i></label>
                    <input type="checkbox" id="rp_subsample_msa" onchange="toggleNumSubsampled(this)">
                  </div>
                   <div class="row" id="num_subsampled_msa_row" style="display:none;">
                      <label for="rp_num_subsampled_msa">Number of Subsampled MSA <i class="tooltip-icon">?<span class="tooltip-text">The number of sequences to use when subsampling the MSA.</span></i></label>
                      <select id="rp_num_subsampled_msa">
                          <option>4</option><option>8</option><option>16</option><option>32</option><option>64</option>
                          <option>128</option><option>256</option><option>512</option><option selected>1024</option>
                      </select>
                  </div>
                  <div class="row">
                      <label for="rp_msa_pairing_strategy">MSA Pairing Strategy <i class="tooltip-icon">?<span class="tooltip-text">Strategy for pairing sequences in the MSA. 'greedy' is faster, 'complete' can be more thorough.</span></i></label>
                      <select id="rp_msa_pairing_strategy">
                          <option>greedy</option><option>complete</option>
                      </select>
                  </div>
              </div>
          </details>
      </div>
      <div class="controls">
          <button class="btn secondary" onclick="showMainParams()"><i class="fa-solid fa-arrow-left"></i> Back</button>
          <button class="btn success" onclick="saveRunParams()"><i class="fa-solid fa-check"></i> OK</button>
          <div id="run_status"></div>
      </div>
    </div>

    <template id="first_protein_template">
        <div class="block seq-block first-protein" data-type="protein">
          <div class="block-header"><div class="title"><i class="fa-solid fa-dna"></i>Protein (Primary)</div></div>
          <div class="row"><label>IDs (comma):</label><input class="p-ids" type="text" placeholder="A,B" oninput="formatIDs(this)"/></div>
          <div class="row"><label>Sequence:</label><textarea class="p-seq" rows="5" style="text-transform: uppercase;"></textarea></div>
        </div>
    </template>

    <template id="protein_template">
        <div class="block seq-block" data-type="protein">
          <div class="block-header">
            <div class="title"><i class="fa-solid fa-dna"></i>Protein</div>
            <button class="remove-btn" onclick="removeBlock(this)" title="Remove block"><i class="fa-solid fa-trash-can"></i></button>
          </div>
          <div class="row"><label>IDs (comma):</label><input class="p-ids" type="text" placeholder="C,D" oninput="formatIDs(this)"/></div>
          <div class="row"><label>Sequence:</label><textarea class="p-seq" rows="5" style="text-transform: uppercase;"></textarea></div>
        </div>
    </template>

    <template id="ligand_template">
        <div class="block seq-block" data-type="ligand">
          <div class="block-header">
            <div class="title"><i class="fa-solid fa-puzzle-piece"></i>Ligand</div>
            <button class="remove-btn" onclick="removeBlock(this)" title="Remove block"><i class="fa-solid fa-trash-can"></i></button>
          </div>
          <div class="row"><label>IDs (comma):</label><input class="l-ids" type="text" placeholder="E,F" oninput="formatIDs(this); updateLigandChainSelector();"/></div>
          <div class="row"><label>Type:</label>
            <select class="l-type" onchange="onLigandTypeChange(this)">
              <option value="ccd">CCD</option><option value="smiles">SMILES</option>
            </select>
          </div>
          <div class="row lig-value-row"><label>Value:</label><input class="l-value" style="text-transform: uppercase;" type="text" placeholder="e.g., SAH" /></div>
        </div>
    </template>
</div>

<script>
  const container = document.getElementById('sequences_container');

  function showRunParams() {
      document.getElementById('main_params_container').style.display = 'none';
      document.getElementById('run_params_container').style.display = 'block';
  }
  function showMainParams() {
      document.getElementById('run_params_container').style.display = 'none';
      document.getElementById('main_params_container').style.display = 'block';
      document.getElementById('run_status').innerHTML = '';
  }
  function toggleNumSubsampled(checkbox) {
      const row = document.getElementById('num_subsampled_msa_row');
      row.style.display = checkbox.checked ? 'flex' : 'none';
  }

  async function saveRunParams() {
      const getVal = id => document.getElementById(id).value;
      const getChecked = id => document.getElementById(id).checked;

      const content = `job_name = "${getVal('rp_job_name')}"
use_potentials = ${getChecked('rp_use_potentials')}
override = ${getChecked('rp_override')}
recycling_steps = ${getVal('rp_recycling_steps')}
sampling_steps = ${getVal('rp_sampling_steps')}
diffusion_samples = ${getVal('rp_diffusion_samples')}
step_scale = ${getVal('rp_step_scale')}
max_msa_seqs = ${getVal('rp_max_msa_seqs')}
subsample_msa = ${getChecked('rp_subsample_msa')}
num_subsampled_msa = ${getVal('rp_num_subsampled_msa')}
msa_pairing_strategy = "${getVal('rp_msa_pairing_strategy')}"`;

      const payload = { filename: 'run_params.txt', content: content.trim() };
      const runStatusEl = document.getElementById('run_status');

      try {
          const result = await google.colab.kernel.invokeFunction('save_run_params', [payload], {});
          if (result && result.status === 'ok') {
              runStatusEl.innerHTML = `<div class="status-message success"><i class="fa-solid fa-check-circle"></i> Parameters saved successfully, you can run <strong>BoltzEngine</strong> now.</div>`;
          } else {
              runStatusEl.innerHTML = `<div class="status-message error"><i class="fa-solid fa-circle-xmark"></i> <strong>Error:</strong> ${result?.message || 'Unknown error.'}</div>`;
          }
      } catch (err) {
          runStatusEl.innerHTML = `<div class="status-message error"><i class="fa-solid fa-circle-xmark"></i> <strong>Save failed:</strong> ${err.toString()}</div>`;
      }
  }

  function formatIDs(inputElement) {
    const originalValue = inputElement.value;
    const formattedValue = originalValue.replace(/[\s,]+/g, '').split('').join(',');
    inputElement.value = formattedValue.toUpperCase();
  }

  function addBlock(templateId) {
      const tpl = document.getElementById(templateId);
      const node = tpl.content.cloneNode(true);
      container.appendChild(node);
  }

  function addProtein(first=false) { addBlock(first ? 'first_protein_template' : 'protein_template'); }

  function addLigand() {
      addBlock('ligand_template');
      if (!document.getElementById('affinity-prediction-section')) {
          const affinityContainer = document.getElementById('affinity_prediction_container');
          affinityContainer.innerHTML = `
            <div id="affinity-prediction-section" class="block" style="border-top-color: var(--secondary-color); margin-bottom: 0;">
                <div class="row" style="align-items: center; margin-bottom: 12px;">
                    <input type="checkbox" id="predict_affinity_toggle" onchange="toggleAffinityOptions(this)" style="width: auto; flex: 0; height: 16px; width: 16px; cursor: pointer;">
                    <label for="predict_affinity_toggle" style="width: auto; cursor: pointer; color: var(--text-dark); font-weight: 500;">Predict Ligand Affinity</label>
                </div>
                <div id="ligand_chain_selector_container" style="display:none; margin-top: 10px;" class="row">
                    <label for="ligand_chain_id_select">Ligand Chain:</label>
                    <select id="ligand_chain_id_select"></select>
                </div>
            </div>`;
      }
  }

  function toggleAffinityOptions(checkbox) {
      const selectorContainer = document.getElementById('ligand_chain_selector_container');
      if (checkbox.checked) {
          selectorContainer.style.display = 'flex';
          updateLigandChainSelector();
      } else {
          selectorContainer.style.display = 'none';
      }
  }

  function updateLigandChainSelector() {
      const selector = document.getElementById('ligand_chain_id_select');
      if (!selector) return;
      const currentVal = selector.value;
      selector.innerHTML = '';
      const allLigandIDs = new Set();
      document.querySelectorAll('.seq-block[data-type="ligand"] .l-ids').forEach(input => {
          (input.value || '').split(',').map(s => s.trim()).filter(Boolean).forEach(id => allLigandIDs.add(id));
      });

      if (allLigandIDs.size === 0) {
          const option = document.createElement('option');
          option.textContent = 'No ligand IDs defined';
          option.value = '';
          selector.appendChild(option);
      } else {
          allLigandIDs.forEach(id => {
              const option = document.createElement('option');
              option.value = id;
              option.textContent = id;
              selector.appendChild(option);
          });
      }
      if (allLigandIDs.has(currentVal)) { selector.value = currentVal; }
  }

  function removeBlock(btn) {
      btn.closest('.seq-block')?.remove();
      if (document.querySelectorAll('.seq-block[data-type="ligand"]').length === 0) {
          document.getElementById('affinity_prediction_container').innerHTML = '';
      } else {
          updateLigandChainSelector();
      }
  }

  function clearAll() {
      container.querySelectorAll('.seq-block:not(.first-protein)').forEach(el => el.remove());
      const first = container.querySelector('.first-protein');
      if (first) {
          first.querySelectorAll('input, textarea').forEach(el => el.value = '');
      }
      document.getElementById('affinity_prediction_container').innerHTML = '';
      document.getElementById('status').innerHTML = '';
      document.getElementById('nextBtn').style.display = 'none';
  }

  function onLigandTypeChange(select) {
      const valueInput = select.closest('.seq-block').querySelector('.l-value');
      valueInput.placeholder = select.value === 'ccd' ? 'e.g., SAH' : 'e.g., CCO... (SMILES)';
  }

  function setStatus(message, type) {
      const statusEl = document.getElementById('status');
      const icon = { success: 'fa-check-circle', error: 'fa-circle-xmark', warning: 'fa-triangle-exclamation'}[type] || 'fa-circle-info';
      statusEl.innerHTML = `<div class="status-message ${type}"><i class="fa-solid ${icon}"></i> ${message}</div>`;
      document.getElementById('nextBtn').style.display = (type === 'success') ? 'inline-flex' : 'none';
  }

  async function saveYaml() {
      const saveBtn = document.getElementById('saveBtn');
      const saveIcon = document.getElementById('saveIcon');
      const saveBtnText = document.getElementById('saveBtnText');
      setStatus('Validating...', 'warning');
      const sequences = [];
      const blocks = document.querySelectorAll('.seq-block');
      const allIDs = new Set();
      let valid = true;

      for (const [idx, b] of Array.from(blocks).entries()) {
          const type = b.dataset.type;
          let currentIds = [];

          if (type === 'protein') {
              currentIds = (b.querySelector('.p-ids').value || '').split(',').map(s => s.trim()).filter(Boolean);
              const seq = b.querySelector('.p-seq').value.trim();
              if (currentIds.length === 0 || !seq) {
                  valid = false; setStatus(`<strong>Error:</strong> Protein block ${idx + 1} requires both IDs and a Sequence.`, 'error'); break;
              } else {
                  sequences.push({ protein: { id: currentIds, sequence: seq } });
              }
          } else if (type === 'ligand') {
              currentIds = (b.querySelector('.l-ids').value || '').split(',').map(s => s.trim()).filter(Boolean);
              const ltype = b.querySelector('.l-type').value;
              const lvalue = b.querySelector('.l-value').value.trim();
              if (currentIds.length === 0 || !lvalue) {
                  valid = false; setStatus(`<strong>Error:</strong> Ligand block ${idx + 1} requires both IDs and a Value.`, 'error'); break;
              } else {
                  const entry = { id: currentIds };
                  if (ltype === 'ccd') entry.ccd = lvalue; else entry.smiles = lvalue;
                  sequences.push({ ligand: entry });
              }
          }
          for (const id of currentIds) {
              if (allIDs.has(id)) {
                  valid = false; setStatus(`<strong>Error:</strong> Duplicate ID '<strong>${id}</strong>' found in block ${idx + 1}. IDs must be unique.`, 'error'); break;
              }
              allIDs.add(id);
          }
          if (!valid) break;
      }
      if (!valid) return;

      const payload = { sequences: sequences };
      const predictAffinityCheckbox = document.getElementById('predict_affinity_toggle');
      if (predictAffinityCheckbox && predictAffinityCheckbox.checked) {
          const selectedLigandId = document.getElementById('ligand_chain_id_select').value;
          if (selectedLigandId) {
              payload.properties = [{ affinity: { binder: selectedLigandId } }];
          } else {
              setStatus('<strong>Error:</strong> "Predict Ligand Affinity" is checked, but no ligand chain is selected or defined.', 'error'); return;
          }
      }

      saveBtn.disabled = true;
      saveBtnText.innerText = 'Saving...';
      saveIcon.className = 'fa-solid fa-spinner fa-spin';
      try {
          const result = await google.colab.kernel.invokeFunction('save_params', [payload], {});
          if (result && result.status === 'ok') {
              setStatus(`Parameter File Saved Successfully`, 'success');
          } else {
              setStatus(`<strong>Error:</strong> ${result?.message || 'Unknown error occurred.'}`, 'error');
          }
      } catch (err) {
          setStatus(`<strong>Save failed:</strong> ${err.toString()}`, 'error');
      } finally {
          saveBtn.disabled = false;
          saveBtnText.innerText = 'Save YAML';
          saveIcon.className = 'fa-solid fa-save';
      }
  }

  addProtein(true); // Initialize UI
</script>
"""

display(HTML(html))

In [ ]:
# @title Boltz2 Engine
import sys
import threading
import time
import os
import re
import shutil
import numpy as np
import matplotlib.pyplot as plt
from Bio.PDB import MMCIFParser, PDBIO
import py3Dmol
import subprocess
import io
import base64
from IPython.display import display, HTML

# --- Helper Functions ---

# ANSI color codes for colored output
class Color:
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    BLUE = "\033[94m"
    MAGENTA = "\033[95m"
    RESET = "\033[0m"

def loader(msg, stop_event):
    """Displays a CLI loading animation."""
    symbols = ["⠋","⠙","⠹","⠸","⠼","⠴","⠦","⠧","⠇","⠏"]
    i = 0
    while not stop_event.is_set():
        sys.stdout.write(f"\r[{symbols[i % len(symbols)]}] {msg}   ")
        sys.stdout.flush()
        time.sleep(0.1)
        i += 1
    sys.stdout.write("\r" + " " * (len(msg) + 10) + "\r")
    sys.stdout.flush()

def parse_value(value_str):
    """Converts a string value from the params file to the appropriate Python type."""
    value_str = value_str.strip()
    if value_str.lower() == 'true': return True
    if value_str.lower() == 'false': return False
    if value_str.startswith('"') and value_str.endswith('"'): return value_str[1:-1]
    try:
        return float(value_str) if '.' in value_str else int(value_str)
    except ValueError:
        return value_str

def clean_ansi_codes(text):
    """Removes ANSI escape sequences from a string."""
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

# --- Visualization Function (Modified) ---

def create_visualizations(job_name, model_id=0, b_min=50, b_max=90):
    """
    Generates only the 3D viewer HTML and returns the PDB data.
    """
    base_path = f"/content/boltz_data/{job_name}/boltz_results_{job_name}/predictions/{job_name}"
    pdb_file   = f"{base_path}/{job_name}_model_{model_id}.pdb"

    # --- Load PDB Data ---
    with open(pdb_file, "r") as f:
        pdb_data = f.read()

    # The py3Dmol viewer is embedded directly in the HTML template for dynamic control
    # We just need to return the PDB data to be inserted into the JS
    return {
        "pdb_data": pdb_data,
    }


# --- Main Script Logic ---

# 1. Set up parameters
os.chdir("/content/boltz_data/")
params_filepath = "/content/boltz_data/run_params.txt"
params = {}
with open(params_filepath, 'r') as f:
    for line in f:
        if '=' in line:
            key, value_str = line.split('=', 1)
            params[key.strip()] = parse_value(value_str)

# Assign parameters to variables
job_name = params.get("job_name", "boltz2_job")
use_potentials = params.get("use_potentials", False)
override = params.get("override", False)
recycling_steps = params.get("recycling_steps", 3)
sampling_steps = params.get("sampling_steps", 50)
diffusion_samples = params.get("diffusion_samples", 1)
step_scale = params.get("step_scale", 10.0)
max_msa_seqs = params.get("max_msa_seqs", 254)
msa_pairing_strategy = params.get("msa_pairing_strategy", "unpaired_paired")

# 2. Prepare directory and parameter file
output_path = f"/content/boltz_data/{job_name}"
if os.path.exists(output_path):
    shutil.rmtree(output_path)

source_file = '/content/boltz_data/params.yaml'
param_file = f'/content/boltz_data/{job_name}.yaml'
if os.path.exists(source_file):
    sed_command = f"sed '/sequence: |-/ {{ N; s/|-\\n\\s*/ / }}' {source_file} > {param_file}"
    subprocess.run(sed_command, shell=True, check=True)
else:
    if not os.path.exists(param_file):
        raise FileNotFoundError(f"Cannot proceed: The parameter file '{param_file}' does not exist.")

# 3. Construct and run the Boltz2 command
cmd = [
    "boltz", "predict", param_file, "--use_msa_server", "--out_dir", job_name,
    "--recycling_steps", str(recycling_steps), "--sampling_steps", str(sampling_steps),
    "--diffusion_samples", str(diffusion_samples), "--step_scale", str(step_scale),
    "--max_msa_seqs", str(max_msa_seqs), "--msa_pairing_strategy", msa_pairing_strategy,
    "--output_format", "pdb"
]
if use_potentials: cmd.append("--use_potentials")
if override: cmd.append("--override")

# Run with loader animation
stop_event = threading.Event()
t = threading.Thread(target=loader, args=(f"{Color.RESET}Running Boltz2 prediction...", stop_event))
t.start()

job_output_html = ""
job_failed = False
visual_data = None # Renamed from 'visuals'

try:
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    stop_event.set()
    t.join()
    print(f"[{Color.GREEN}✔{Color.RESET}] Boltz2 run finished successfully!")
    # Combine stdout and stderr for full log, clean ANSI codes
    full_output = f"STDOUT:\n{result.stdout}\n\nSTDERR:\n{result.stderr}"
    job_output_html = f'<pre class="output-box success">{clean_ansi_codes(full_output)}</pre>'

    # Generate visualizations on success
    pdb_to_check = f"/content/boltz_data/{job_name}/boltz_results_{job_name}/predictions/{job_name}/{job_name}_model_0.pdb"
    if os.path.exists(pdb_to_check):
        visual_data = create_visualizations(job_name=job_name, model_id=0)
    else:
        job_output_html += f'<pre class="output-box error">Error: Output PDB not found at {pdb_to_check}</pre>'

except subprocess.CalledProcessError as e:
    job_failed = True
    stop_event.set()
    t.join()
    print(f"[{Color.RED}✘{Color.RESET}] Boltz2 run failed. See details in the HTML output below.")
    # Format error output
    error_output = clean_ansi_codes(e.stderr)
    job_output_html = f'<h2>Job Failed</h2><pre class="output-box error">Exit Code: {e.returncode}\n\n{error_output}</pre>'


# 4. Generate and display the final HTML output
pdb_string_for_js = ""
if visual_data and "pdb_data" in visual_data:
    pdb_string_for_js = visual_data['pdb_data'].replace('\\', '\\\\').replace('`', '\\`').replace('$', '\\$')


html_template = """
<style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto+Mono&family=Roboto:wght@400;500;700&display=swap');
    .boltz-container {{
        font-family: 'Roboto', sans-serif;
        background-color: #ffffff;
        color: #212121;
        border: 1px solid #e0e0e0;
        border-radius: 10px;
        padding: 20px;
        margin: 10px;
        box-shadow: 0 4px 8px rgba(0,0,0,0.05);
    }}
    .boltz-container h1, .boltz-container h2, .boltz-container h3 {{
        font-family: 'Roboto', sans-serif;
        color: #0d47a1; /* Dark Blue */
        border-bottom: 2px solid #0d47a1;
        padding-bottom: 5px;
        margin-top: 20px;
    }}
    .boltz-container h1 {{
        text-align: center;
        font-size: 2em;
        font-weight: 700;
        color: #004d40; /* Dark Teal */
        border-bottom: none;
    }}
    .boltz-container .job-name-span {{
        font-family: 'Roboto Mono', monospace;
        background-color: #eeeeee;
        color: #bf360c; /* Dark Orange */
        padding: 3px 8px;
        border-radius: 5px;
        font-weight: bold;
    }}
    .output-box {{
        background-color: #f5f5f5;
        border: 1px solid #e0e0e0;
        border-radius: 5px;
        padding: 15px;
        white-space: pre-wrap;
        word-wrap: break-word;
        max-height: 400px;
        overflow-y: auto;
        font-family: 'Roboto Mono', monospace;
        font-size: 0.9em;
        color: #333;
    }}
    .output-box.success {{ border-left: 5px solid #388e3c; }}
    .output-box.error {{ border-left: 5px solid #d32f2f; color: #c62828; }}

    .viz-container {{
        display: flex;
        flex-wrap: wrap; /* Allows items to wrap on smaller screens */
        gap: 20px;
        margin-top: 20px;
    }}
    .viz-options {{
        flex: 1; /* Takes available space */
        min-width: 280px; /* Minimum width before wrapping */
        background-color: #ffffff;
        border: 1px solid #e0e0e0;
        border-radius: 8px;
        padding: 15px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.04);
    }}
    .viz-viewer {{
        flex: 2; /* Takes more space for the viewer */
        min-width: 500px; /* Minimum width for viewer */
        height: 500px; /* Fixed height for the viewer */
        background-color: #f5f5f5;
        border: 1px solid #e0e0e0;
        border-radius: 8px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.04);
        position: relative;
    }}
    .viz-options h3 {{
        color: #0d47a1;
        border-bottom: 1px solid #e0e0e0;
        padding-bottom: 8px;
        margin-bottom: 15px;
    }}
    .viz-options label {{
        display: block;
        margin-bottom: 5px;
        font-weight: 500;
        color: #424242;
    }}
    .viz-options select, .viz-options input[type="number"], .viz-options button {{
        width: calc(100% - 10px);
        padding: 8px;
        margin-bottom: 10px;
        border: 1px solid #ccc;
        border-radius: 4px;
        font-family: 'Roboto', sans-serif;
        font-size: 0.9em;
        background-color: #fff;
        color: #333;
    }}
    .viz-options button {{
        background-color: #1976d2; /* Blue */
        color: white;
        border: none;
        cursor: pointer;
        transition: background-color 0.2s ease;
    }}
    .viz-options button:hover {{
        background-color: #1565c0; /* Darker Blue */
    }}
</style>

<script src="https://3Dmol.org/build/3Dmol-min.js"></script>
<script>
    let viewer = null;
    const pdbData = {pdb_string_for_js};

    function initializeViewer() {{
        const element = document.getElementById('mol_viewer');
        if (element && pdbData) {{
            viewer = $3Dmol.createViewer(element, {{ backgroundColor: 'white' }});
            updateViewer(); // Apply initial settings
        }} else {{
            console.error("Viewer element or PDB data not found.");
        }}
    }}

    document.addEventListener('DOMContentLoaded', initializeViewer);
    setTimeout(initializeViewer, 500);  

    function updateViewer() {{
        if (!viewer) return;

        viewer.clear();
        viewer.addModel(pdbData, "pdb");

        const style = document.getElementById('styleSelect').value;
        const colorScheme = document.getElementById('colorSchemeSelect').value;
        const bMin = parseFloat(document.getElementById('bFactorMin').value);
        const bMax = parseFloat(document.getElementById('bFactorMax').value);

        let styleObj = {{}};
        if (style === 'cartoon') {{
            styleObj = {{
                cartoon: {{
                    colorscheme: {{
                        prop: 'b',
                        gradient: colorScheme,
                        min: bMin,
                        max: bMax
                    }}
                }}
            }};
        }} else if (style === 'sphere') {{
            styleObj = {{
                sphere: {{
                    colorscheme: colorScheme // Simple colorscheme for sphere
                }}
            }};
        }} else if (style === 'stick') {{
            styleObj = {{
                stick: {{
                    colorscheme: colorScheme
                }}
            }};
        }} else if (style === 'line') {{
            styleObj = {{
                line: {{
                    colorscheme: colorScheme
                }}
            }};
        }}


        viewer.setStyle({{}}, styleObj);
        viewer.addStyle({{'hetflag': true}}, {{'stick': {{'colorscheme': 'default'}}}}) // Always show heteroatoms as sticks

        viewer.zoomTo();
        viewer.render();
    }}

    // Initialize viewer after the DOM is fully loaded
    document.addEventListener('DOMContentLoaded', initializeViewer);
</script>

<div class="boltz-container">
    <h1>Boltz2 Results: <span class="job-name-span">{job_name}</span></h1>

    <div class="section">
        <h2>Job Output</h2>
        {job_output_html}
    </div>

    {visualization_html_content}
</div>
"""

visualization_html_content = ""
if visual_data: # Check if visual_data was successfully populated
    visualization_html_content = f"""
    <div class="section">
        <h2>Protein Structure Visualization</h2>
        <div class="viz-container">
            <div class="viz-options">
                <h3>Display Options</h3>
                <div>
                    <label for="styleSelect">Style:</label>
                    <select id="styleSelect" onchange="updateViewer()">
                        <option value="cartoon" selected>Cartoon</option>
                        <option value="sphere">Sphere</option>
                        <option value="stick">Stick</option>
                        <option value="line">Line</option>
                    </select>
                </div>
                <div>
                    <label for="colorSchemeSelect">Color Scheme (pLDDT for Cartoon):</label>
                    <select id="colorSchemeSelect" onchange="updateViewer()">
                        <option value="roygb" selected>Rainbow (pLDDT)</option>
                        <option value="blueWhiteRed">Blue-White-Red</option>
                        <option value="greenCarbon">Green Carbon</option>
                        <option value="default">Default</option>
                    </select>
                </div>
                <div>
                    <label for="bFactorMin">B-Factor Min (for pLDDT coloring):</label>
                    <input type="number" id="bFactorMin" value="50" step="1" onchange="updateViewer()">
                </div>
                <div>
                    <label for="bFactorMax">B-Factor Max (for pLDDT coloring):</label>
                    <input type="number" id="bFactorMax" value="90" step="1" onchange="updateViewer()">
                </div>
                <button onclick="updateViewer()">Apply Changes</button>
                <button onclick="viewer.zoomTo()">Reset Zoom</button>
            </div>
            <div class="viz-viewer">
                <div id="mol_viewer" style="width:100%; height:100%;"></div>
            </div>
        </div>
    </div>
    """

# Render the final HTML
display(HTML(html_template.format(
    job_name=job_name,
    job_output_html=job_output_html,
    pdb_string_for_js=pdb_string_for_js,
    visualization_html_content=visualization_html_content
)))

In [ ]:
# @title Plot Results

base_path = f"/content/boltz_data/{job_name}/boltz_results_{job_name}/predictions/{job_name}"
pdb_file   = f"{base_path}/{job_name}_model_{model_id}.pdb"
plddt_file = f"{base_path}/plddt_{job_name}_model_{model_id}.npz"
pae_file   = f"{base_path}/pae_{job_name}_model_{model_id}.npz"

# --- pLDDT Plot ---
print(f"\n{Color.CYAN}{'='*50}{Color.RESET}")
print(f"{Color.MAGENTA}Predicted Local Distance Difference Test (pLDDT){Color.RESET}")
print(f"{Color.RESET}Confidence score per residue: Higher = more reliable structure.{Color.RESET}")
print(f"{Color.CYAN}{'='*50}{Color.RESET}\n")

plddt = np.load(plddt_file)["plddt"]
plt.figure(figsize=(10,4))
plt.plot(plddt, label="pLDDT", color="blue")
plt.xlabel("Residue index")
plt.ylabel("pLDDT score")
plt.title(f"Model {model_id} | Confidence per residue")
plt.legend()
plt.tight_layout(pad=3.0)
plt.show()

# --- PAE Heatmap ---
print(f"\n{Color.CYAN}{'='*50}{Color.RESET}")
print(f"{Color.MAGENTA}Predicted Aligned Error (PAE) Heatmap{Color.RESET}")
print(f"{Color.RESET}Shows expected positional error between residue pairs.\nLower values = more reliable alignment.{Color.RESET}")
print(f"{Color.CYAN}{'='*50}{Color.RESET}\n")

pae = np.load(pae_file)["pae"]
plt.figure(figsize=(6,5))
plt.imshow(pae, cmap="viridis", origin="lower")
plt.colorbar(label="Predicted Aligned Error (Å)")
plt.title(f"Model {model_id} | PAE Heatmap")
plt.xlabel("Residue index")
plt.ylabel("Residue index")
plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
# @title Copy Results to Drive
import shutil, os
from google.colab import drive
from Bio.PDB import MMCIFParser, PDBIO

# ANSI color codes for colored output
class Color:
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    BLUE = "\033[94m"
    MAGENTA = "\033[95m"
    RESET = "\033[0m"

# Mount Google Drive
drive.mount('/content/drive')

# Paths
drive_output_dir = f"/content/drive/MyDrive/Boltz2_Results/{job_name}"
local_output_path = f"/content/boltz_data/{job_name}"

# # Convert CIF to PDB
# cif_file = f"{local_output_path}/boltz_results_{job_name}/predictions/{job_name}/{job_name}_model_0.cif"
# pdb_file = f"{local_output_path}/{job_name}.pdb"

# parser = MMCIFParser(QUIET=True)
# structure = parser.get_structure("prot", cif_file)
# io = PDBIO()
# io.set_structure(structure)
# io.save(pdb_file)

# Remove old folder in Drive if exists
if os.path.exists(drive_output_dir):
    print(f"Removing existing folder {drive_output_dir}")
    shutil.rmtree(drive_output_dir)
    print("Old Drive folder removed.")

# Copy local output folder to Drive
shutil.copytree(local_output_path, drive_output_dir)
print(f"{Color.GREEN}All results copied to Google Drive: {drive_output_dir}{Color.RESET}")
# # Copy PDB file separately (optional, just in case)
# drive_pdb_file = os.path.join(drive_output_dir, os.path.basename(pdb_file))
# shutil.copy(pdb_file, drive_pdb_file)

In [ ]:
# @title Download Results (.zip)
from google.colab import files
from Bio.PDB import MMCIFParser, PDBIO
import shutil
import os

# Local output folder you want to download
local_output_path = f"/content/boltz_data/{job_name}"

# # Convert CIF to PDB
# cif_file = f"{local_output_path}/boltz_results_{job_name}/predictions/{job_name}/{job_name}_model_0.cif"
# pdb_file = f"{local_output_path}/{job_name}.pdb"

# # Parse CIF and save as PDB
# parser = MMCIFParser(QUIET=True)
# structure = parser.get_structure("prot", cif_file)
# io = PDBIO()
# io.set_structure(structure)
# io.save(pdb_file)

# Path for the zip file
zip_file = f"/content/{job_name}.zip"

# Remove previous zip if exists
if os.path.exists(zip_file):
    os.remove(zip_file)

# Create zip of the entire folder
shutil.make_archive(base_name=f"/content/{job_name}", format='zip', root_dir=local_output_path)

# Download the zip file
files.download(zip_file)

# Success message
print(f"{Color.GREEN}Download successful! All results from '{job_name}' are saved in '{zip_file}'{Color.RESET}")
